# Tensorboard使用
1. 指定logdir,创建SummaryWriter对象：writer = SummaryWriter("./logs/2_Tensorboard")
3. 添加数据
2. wirter.add_image()
3. writer.add_scalar()
4. writer.close()


控制台输入：tensorboard --logdir=logs --port=6006(可选)

In [41]:
from torch.utils.tensorboard import SummaryWriter
import os
from PIL import Image
import torch
from torchvision import transforms, utils
import math

In [42]:
# 创建SummaryWriter对象，指定日志存储路径
writer = SummaryWriter("./logs/2_Tensorboard")

In [43]:
# 常用步骤为以下三步
# 1. 添加数据
# 2. writer.add_image() 或 writer.add_scalar()
# 3. 关闭writer
# writer.close()

In [44]:
# 如何添加函数图像
# writer.add_scalar()  用于绘制矢量图
    # def add_scalar(
    #     self,
    #     tag,                图表title
    #     scalar_value,       图标数值 y轴
    #     global_step=None,   步数 x轴
    #     walltime=None,
    #     new_style=False,
    #     double_precision=False,
    # ):

for i in range(100):
    # 不同的tag会绘制到不同的表中，不用怕串
    # writer.add_scalar("y = x", i, i)
    writer.add_scalar("y = 2x", 2 * i, i)

In [45]:
# 如何添加图像
# writer.add_image() 用于将图像添加到TensorBoard
    # def add_image(
    #     self,
    #     tag,                 图title
    #     img_tensor,          图数据，需要：torch.Tensor, numpy.ndarray, or string/blobname 类型
    #     global_step=None,
    #     walltime=None,
    #     dataformats="CHW"。  img_tensor默认尺寸为(3, H, W)，3通道H高度W宽度。如果是其他形状需要使用dataformats变量来进行说明。
    # ):

In [46]:
# 定义图像文件夹路径
image_path = "./data/hymenoptera_data/train/ants"

In [47]:
# 定义图像预处理变换：调整为统一大小并转换为Tensor格式
# 这一部分详细内容看3_Transforms
# transforms.Compose用于将多个变换操作组合在一起
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # 调整图片大小
    transforms.ToTensor()           # 转换为Tensor
])

In [48]:
# 获取 image_path 文件夹下的所有文件名
image_files = os.listdir(image_path)
# 用于存储所有图片的Tensor
images_list = []

In [49]:
# 遍历所有文件，将图片打开、预处理后保存到列表中
for file_name in image_files:
    full_path = os.path.join(image_path, file_name)
    # 打开图片，并确保转换为RGB模式（确保一致性）
    img_pil = Image.open(full_path).convert("RGB")
    # 应用预处理变换，将PIL Image转换为Tensor
    img_tensor = transform(img_pil)
    images_list.append(img_tensor)

In [50]:
# 将列表中所有的Tensor堆叠成一个Tensor，形状为 [N, C, H, W]
img = torch.stack(images_list) # 将图片列表堆叠成一个4D Tensor
print(img.shape)

torch.Size([124, 3, 256, 256])


In [51]:
# 定义每个 global_step 下要显示的图片数量
img_num = 9  # 这里可以按需要调整，每个global_step显示的图片数量

In [52]:
# 总图片数
total_images = img.size(0) # 获取图片总数

In [53]:
# 按 img_num 数量对图片进行分组，每个分组对应一个 global_step
num_steps = math.ceil(total_images / img_num) # 计算需要的global_step数量
print(f"需要的 global_step 数量：{num_steps}")

需要的 global_step 数量：14


In [54]:
# 遍历每个global_step，将图片写入TensorBoard
for step in range(num_steps):
    # 计算当前批次的起始和结束索引
    start_idx = step * img_num # 当前批次起始索引
    end_idx = min((step + 1) * img_num, total_images) # 当前批次结束索引

    # 从总图片中切出当前批次
    subset = img[start_idx:end_idx] # 获取当前批次的图片

    # 生成图片网格, nrow 参数可以根据需要调整，这里简单使用 img_num 或计算一个较为合理的行数
    grid = utils.make_grid(subset, nrow=int(math.sqrt(img_num))) # nrow控制每行显示的图片数量

    # 将当前图片网格写入 TensorBoard，并以当前 step 作为 global_step
    writer.add_image("All_Images", grid, global_step=step)

    print(f"写入 global_step = {step}, 图片索引范围：[{start_idx}, {end_idx})")

写入 global_step = 0, 图片索引范围：[0, 9)
写入 global_step = 1, 图片索引范围：[9, 18)
写入 global_step = 2, 图片索引范围：[18, 27)
写入 global_step = 3, 图片索引范围：[27, 36)
写入 global_step = 4, 图片索引范围：[36, 45)
写入 global_step = 5, 图片索引范围：[45, 54)
写入 global_step = 6, 图片索引范围：[54, 63)
写入 global_step = 7, 图片索引范围：[63, 72)
写入 global_step = 8, 图片索引范围：[72, 81)
写入 global_step = 9, 图片索引范围：[81, 90)
写入 global_step = 10, 图片索引范围：[90, 99)
写入 global_step = 11, 图片索引范围：[99, 108)
写入 global_step = 12, 图片索引范围：[108, 117)
写入 global_step = 13, 图片索引范围：[117, 124)


In [55]:
# 关闭SummaryWriter
writer.close()